In [1]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc,os,random
import time,datetime
from tqdm import tqdm


In [12]:

p0 = pd.read_csv('C:/Users/m0815/Downloads/submission/submission (Cat with a lot features).csv')
p1 = pd.read_csv('C:/Users/m0815/Downloads/submission/submission (SpeedCat).csv')
p2 = pd.read_csv('C:/Users/m0815/Downloads/submission/submission (XGB).csv')
p3 = pd.read_csv('C:/Users/m0815/Downloads/submission/submission (cudf).csv')

p0['correct'] = p0['correct']*0.3 + p1['correct']*0.35 + p2['correct']*0.15 + p3['correct']*0.1
for index,row in p0.iterrows():
    # print(row['correct'])
    if float(row['correct'])>=0.5: row['correct']=1
    else: row['correct'] =0
    p0.iloc[index, p0.columns.get_loc('correct')] = row['correct']
p0.to_csv('result.csv',index=False)


In [11]:
import pandas as pd

# 创建一个DataFrame
data = {'name': ['Alice', 'Bob', 'Charlie'], 'age': [25, 30, 35]}
df = pd.DataFrame(data)

# 遍历每行并更改'age'列的值
for index, row in df.iterrows():
    # 计算新的年龄值
    new_age = row['age'] + 1
    
    # 在DataFrame中使用.iloc属性更改该行的'age'列的值
    df.iloc[index, df.columns.get_loc('age')] = new_age

# 打印更新后的DataFrame
print(df)


      name  age
0    Alice   26
1      Bob   31
2  Charlie   36


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

# 定义三个模型的预测结果
pred_speedcat = [0.1, 0.2, 0.3, 0.4]
pred_xgb = [0.3, 0.2, 0.1, 0.4]
pred_cudf = [0.2, 0.1, 0.4, 0.3]

# 定义加权系数的步长和范围
step_size = 0.1
weights = np.arange(step_size, 1.0+step_size, step_size)

# 定义评分函数
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# 构造参数字典
params = {
    'weights': [list(w) for w in np.ndindex((len(weights),)*3)],
}

# 定义模型
model = GridSearchCV(
    estimator=WeightedModel([speedcat, xgb, cudf]),
    param_grid=params,
    scoring=make_scorer(rmse, greater_is_better=False),
    cv=5,
    n_jobs=-1
)

# 训练模型
model.fit(X_train, y_train)

# 输出最优参数组合及其对应的性能指标
print(f"Best parameters: {model.best_params_}")
print(f"RMSE score: {-model.best_score_:.4f}")

# 预测结果并输出性能指标
y_pred = model.predict(X_test)
print(f"Test RMSE score: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")

# WeightedModel 定义如下
class WeightedModel:
    def __init__(self, models):
        self.models = models
    
    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)
    
    def predict(self, X):
        preds = np.array(
            [model.predict(X) for model in self.models]
        )
        return (preds.T * self.weights).sum(axis=1)
